### 리뷰 평점이 셀러의 평점에 얼마나 영향을 끼치는지 ** ????

In [ ]:
# 필요한 라이브러리 로드

import pandas as pd



In [ ]:
# 데이터 로드

reviews = pd.read_csv('../reviews.csv')
orders = pd.read_csv('../orders.csv')
order_items = pd.read_csv('../order_items.csv')
products = pd.read_csv('../products.csv')

# 데이터 첫 줄 및 컬럼 확인

reviews.head(), orders.head()



In [ ]:
# Merge reviews with orders based on Order_id
reviews_orders_merged = pd.merge(reviews, orders, on='Order_id')

# Merge this with order_items to get seller information
complete_data = pd.merge(reviews_orders_merged, order_items, on='Order_id')

# Calculate total revenue and average review score per seller
seller_performance = complete_data.groupby('Seller_id').agg({
    'Price': 'sum',  # Total revenue
    'Review_score': 'mean'  # Average review score
}).reset_index()

# Rename the columns for clarity
seller_performance.columns = ['Seller_id', 'Total_Revenue', 'Average_Review_Score']

# Display a few rows and calculate correlation
correlation = seller_performance[['Total_Revenue', 'Average_Review_Score']].corr()
seller_performance.head(), correlation
d

In [ ]:
# 데이터 첫 줄 및 컬럼 확인

reviews.head(), orders.head()

# Merge reviews with orders based on Order_id
reviews_orders_merged = pd.merge(reviews, orders, on='Order_id')

# Merge this with order_items to get seller information
complete_data = pd.merge(reviews_orders_merged, order_items, on='Order_id')

# Calculate total revenue and average review score per seller
seller_performance = complete_data.groupby('Seller_id').agg({
    'Price': 'sum',  # Total revenue
    'Review_score': 'mean'  # Average review score
}).reset_index()

# Rename the columns for clarity
seller_performance.columns = ['Seller_id', 'Total_Revenue', 'Average_Review_Score']

# Display a few rows and calculate correlation
correlation = seller_performance[['Total_Revenue', 'Average_Review_Score']].corr()
seller_performance.head(), correlation
d

셀러별 총 매출과 평균 리뷰 점수를 분석한 결과, 리뷰 점수와 셀러의 매출 사이에는 거의 상관관계가 없는 것으로 나타났습니다(-0.0205). 이는 리뷰 점수가 높다고 해서 반드시 매출이 높은 것은 아니며, 매출이 많은 셀러의 리뷰 점수가 반드시 높은 것도 아니라는 것을 의미합니다.

이 결과는 리뷰 점수가 셀러의 매출에 직접적인 영향을 미치지 않을 수 있음을 시사합니다. 다만, 이는 다양한 요인에 의해 영향을 받을 수 있으므로, 추가적인 요인들을 고려하는 더 광범위한 분석이 필요할 수 있습니다.

In [6]:
# 낮은 리뷰가 달린 뒤에도. 추가적으로 그 상품을 구매하는 사람이 많은지(비싼 상품일 경우)

# Filter reviews with low scores (3 or less)
low_score_reviews = reviews[reviews['Review_score'] <= 3]

# Merge low score reviews with orders to track the order dates
low_score_reviews_orders = pd.merge(low_score_reviews, orders, on='Order_id')

# Merge this data with order_items to track product sales after low reviews
low_score_product_sales = pd.merge(low_score_reviews_orders, order_items, on='Order_id')

# Merge with products to get price information
low_score_product_sales = pd.merge(low_score_product_sales, products, on='Product_id')

# Filter to consider only products with prices in the upper quartile (expensive products)
price_threshold = low_score_product_sales['Price'].quantile(0.75)
expensive_low_score_sales = low_score_product_sales[low_score_product_sales['Price'] >= price_threshold]

# Sort by order date to analyze the trend
expensive_low_score_sales_sorted = expensive_low_score_sales.sort_values(by='Order_purchase_timestamp')

# Convert Order_purchase_timestamp to datetime
orders['Order_purchase_timestamp'] = pd.to_datetime(orders['Order_purchase_timestamp'])

# Group by product and date to see the sales pattern after receiving low scores
sales_after_low_reviews = expensive_low_score_sales_sorted.groupby(['Product_id', expensive_low_score_sales_sorted['Order_purchase_timestamp'].dt.date]).agg({
    'Order_item_id': 'sum'  # Sum up the quantity sold per product per day
}).reset_index()

sales_after_low_reviews.head(10)


AttributeError: Can only use .dt accessor with datetimelike values